One of the features of lumopt GL is the streamlined handling of multipolygon optimization.

Combining Polygons creates a Geometry. A Geometry is defined by an operation. LumoptGL supports 3 operations.

Add: Use this when two or more polygons all have unique/independent parameters. (special method +)
Mul: Use this when two or more polygons all share the same parameters (special method *)
Merge: Use this when two or more polygons share some parameters but not others. A little bit more complicated to use as it expects a mapping.

In [1]:
from config import Config
import os, sys
sys.path.append(Config.LUMERICAL_API_PATH) # if this fails adjust the config.py and add the lumerical api path
root_path = Config.ROOT_DIR
result_path = Config.RESULTS_PATH

In [2]:
import numpy as np
import scipy as sp

import lumapi
from lumopt.utilities.wavelengths import Wavelengths
import lumopt.utilities.wavelengths as w1
from lumopt.geometries.polygon import FunctionDefinedPolygon
from lumopt.utilities.materials import Material
from lumopt.figures_of_merit.modematch import ModeMatch
from lumopt.optimizers.generic_optimizers import ScipyOptimizers
from lumopt.optimization import Optimization
from lumopt.geometries.polygon import FunctionDefinedPolygon
from lumopt.geometries.geometry import Geometry
from lumopt_gl.geometries.polygon_gl import FunctionDefinedPolygon_GL
from lumopt_gl.geometries.geometry_gl import Geometry_GL
from lumopt_gl.optimizers.optimizer_gl import OptimizerGL
from lumopt_gl.optimization import OptimizationGL

CONFIGURATION FILE {'root': 'C:\\Program Files/Lumerical/v232/api/python', 'lumapi': 'C:/Program Files/Lumerical/v232/api/python'}


In this example we will keep things simple. A 1 um taper from 350 to 750 nm is then followed by a 1 um taper back to 350 nm. 

Obviously this is just to understand ADD and MUL conceptually.

For practical applications, please see the SWG geometries in the geometry folder.

Once again, we start by creating the base script. We can use the base_script function from the geometries folder to generate this basescript with our unique arguments for a taper. Poly_wg = True prevents the base script from creating input and output waveguides. 

In [3]:
from geometries.taper import taper_base_script
start_y = 0.175e-6
stop_y = 0.175e-6 # 350 nm to 350 nm output wg.
start_x = -1e-6
stop_x = 1e-6

base_script = taper_base_script(dimension = 2, start_y = start_y, stop_y = stop_y, start_x = start_x, stop_x = stop_x, poly_wg = True) # length 2um, width = 350 to 750 nm

In [4]:
eps_in = Material(name = 'Si: non-dispersive')
eps_out = Material(name = 'SiO2: non-dispersive')

We can create the input and output waveguides as function defined polygons instead. Polygons do not need to have parameters. 

In [5]:
def input_wg(params = None):
    return np.array([(start_x-2e-6, start_y), (start_x-2e-6, -start_y), (start_x, -start_y), (start_x, start_y)])
def output_wg(params=None):
    return np.array([(stop_x + 2e-6, stop_y), (stop_x, stop_y), (stop_x, -stop_y), (stop_x + 2e-6, -stop_y)])

input_wg_poly = FunctionDefinedPolygon_GL(func=input_wg, initial_params=np.empty(0), bounds=np.empty((0, 2)), z=0.0, depth=220e-9, eps_out=eps_out, eps_in=eps_in)
output_wg_poly = FunctionDefinedPolygon_GL(func=output_wg, initial_params=np.empty(0), bounds=np.empty((0, 2)), z=0.0, depth=220e-9, eps_out=eps_out, eps_in=eps_in)

Now we'll make the two tapers.

In [6]:
initial_points_x = np.linspace(start_x, 0, 7) # only go halfway. the symmetrical taper will reflect across the y axis and continue to stop_x
initial_points_y = np.linspace(start_y, 0.375e-6, 7)

init_params_y = initial_points_y[1:-1]

def taper(params = init_params_y):
    ''' Defines a taper where the paramaters are the y coordinates of the nodes of a cubic spline. '''
    points_x = np.concatenate(([initial_points_x.min()], initial_points_x[1:-1], [initial_points_x.max()]))
    points_y = np.concatenate(([initial_points_y[0]], params, [initial_points_y[-1]]))
    
    polygon_points_x = np.linspace(min(points_x), max(points_x), 100)
    interpolator = sp.interpolate.interp1d(points_x, points_y, kind = 'cubic')
    polygon_points_y = interpolator(polygon_points_x)
    
    polygon_points_up = [(x, y) for x, y in zip(polygon_points_x, polygon_points_y)]
    polygon_points_down = [(x, -y) for x, y in zip(polygon_points_x, polygon_points_y)]
    polygon_points = np.array(polygon_points_up[::-1] + polygon_points_down)
    return polygon_points

def symmetrical_taper(params = init_params_y):
    polygon_points = taper(params)
    reflected_points = [(-x, y) for x, y in polygon_points]
    # Reverse the list to maintain CCW orientation
    reflected_points.reverse()
    return np.array(reflected_points)

bounds = [(0.2e-6,0.8e-6)] * len(init_params_y)

depth = 220.0e-9
taper_polygon = FunctionDefinedPolygon_GL(func = taper, initial_params = init_params_y, bounds = bounds, z = 0.0,depth = depth,eps_out = eps_out, eps_in = eps_in, dx = 2e-9)
taper_polygon_symmetry = FunctionDefinedPolygon_GL(func = symmetrical_taper, initial_params = init_params_y, bounds = bounds, z = 0.0,depth = depth,eps_out = eps_out, eps_in = eps_in, dx = 2e-9)

 Since they are symmetrical, they share the same parameters. Therefore these two polygons will be joined to one geometry with the MUL operation.

In [ ]:
taper_geo = Geometry_GL([taper_polygon, taper_polygon_symmetry], 'mul')

We will now add the two waveguides to this geo. Technically, those have no parameters, but we will still use add.

In [ ]:
geometry = Geometry_GL([taper_geo, input_wg_poly, output_wg_poly], 'add')

Define the same optimizer as before, then run

In [ ]:

def optimizer3D(polygon, base_script, max_iter = 40):
    wavelengths = Wavelengths(start = 1500e-9, stop = 1600e-9, points = 11)

    fom = ModeMatch(monitor_name = 'fom',mode_number = 'fundamental TE mode', direction = 'Forward', 
                    target_T_fwd = lambda wl: np.ones(wl.size), norm_p = 1) # for 3D, we need to specify TE/TM

    optimizer = OptimizerGL(max_iter = max_iter, ftol = 1.0e-5, method = 'LD_MMA')

    opt = OptimizationGL(base_script=base_script, wavelengths = wavelengths, fom = fom, geometry = polygon, optimizer = optimizer, 
                       use_var_fdtd=False, store_all_simulations = False, GPU = True) # and set varFDTD to False. Set GPU to True if you have one.
    return opt

In [ ]:
opt = optimizer3D(geometry, base_script)
opt.run()

This time we'll use add to form the geometry from the two tapers.

In [ ]:
from geometries.taper import taper_base_script
stop_y = 0.350e-6 # 350 nm to 350 nm output wg.
base_script = taper_base_script(dimension = 3, start_y = start_y, stop_y = stop_y, start_x = start_x, stop_x = stop_x) # length 2um, width = 350 to 750 nm

In [ ]:
initial_points_x = np.linspace(start_x, 0, 7) # only go halfway. the symmetrical taper will reflect across the y axis and continue to stop_x
initial_points_y = np.linspace(start_y, 0.25e-6, 7)
initial_points_y2 = np.linspace(0.25e-6, 0.35e-6, 7)
initial_points_x2 = np.linspace(0, stop_x, 7) # only go halfway. the symmetrical taper will reflect across the y axis and continue to stop_x

init_params_y = initial_points_y[1:-1]
init_params_y2 = initial_points_y2[1:-1]

def taper(params = init_params_y):
    ''' Defines a taper where the paramaters are the y coordinates of the nodes of a cubic spline. '''
    points_x = np.concatenate(([initial_points_x.min()], initial_points_x[1:-1], [initial_points_x.max()]))
    points_y = np.concatenate(([initial_points_y[0]], params, [initial_points_y[-1]]))
    
    polygon_points_x = np.linspace(min(points_x), max(points_x), 100)
    interpolator = sp.interpolate.interp1d(points_x, points_y, kind = 'cubic')
    polygon_points_y = interpolator(polygon_points_x)
    
    polygon_points_up = [(x, y) for x, y in zip(polygon_points_x, polygon_points_y)]
    polygon_points_down = [(x, -y) for x, y in zip(polygon_points_x, polygon_points_y)]
    polygon_points = np.array(polygon_points_up[::-1] + polygon_points_down)
    return polygon_points

def taper2(params = init_params_y2):
    ''' Defines a taper where the paramaters are the y coordinates of the nodes of a cubic spline. '''
    points_x = np.concatenate(([initial_points_x2.min()], initial_points_x2[1:-1], [initial_points_x2.max()]))
    points_y = np.concatenate(([initial_points_y2[0]], params, [initial_points_y2[-1]]))
    
    polygon_points_x = np.linspace(min(points_x), max(points_x), 100)
    interpolator = sp.interpolate.interp1d(points_x, points_y, kind = 'cubic')
    polygon_points_y = interpolator(polygon_points_x)
    
    polygon_points_up = [(x, y) for x, y in zip(polygon_points_x, polygon_points_y)]
    polygon_points_down = [(x, -y) for x, y in zip(polygon_points_x, polygon_points_y)]
    polygon_points = np.array(polygon_points_up[::-1] + polygon_points_down)
    return polygon_points

bounds = [(0.1e-6,0.8e-6)] * len(init_params_y)

depth = 220.0e-9
taper_polygon = FunctionDefinedPolygon_GL(func = taper, initial_params = init_params_y, bounds = bounds, z = 0.0,depth = depth,eps_out = eps_out, eps_in = eps_in, dx = 2e-9)
taper_polygon_symmetry = FunctionDefinedPolygon_GL(func = taper2, initial_params = init_params_y2, bounds = bounds, z = 0.0,depth = depth,eps_out = eps_out, eps_in = eps_in, dx = 2e-9)

In [ ]:
geo = Geometry_GL([taper_polygon, taper_polygon_symmetry, input_wg_poly, output_wg_poly], 'add')

In [ ]:
opt = optimizer3D(geo, base_script)
opt.run()

In [3]:
from geometries.SWG_to_strip import SWG_to_strip

SWG_finger, base_script = SWG_to_strip(dimension = 2, num_gratings = 20, static_gratings = 3)


: 